# Project 2 - ETL
## American Professional Football Database
The database contains games stats and betting line information from 2010 - 2019.  Sports gambling has exploded over the past few years.  The purpose of this database is to keep track of football game and betting results.  This may evenutally lead to tracking future trends in the sports gambling arena.
## Data Sources
* NFL Games Stats
    
    https://www.kaggle.com/datasets/423c766b86dcb18c859cfd3a4d61e6531df8c488b59839931061fc9b3820c068
* NFL scores and betting data
    
    https://www.kaggle.com/datasets/tobycrabtree/nfl-scores-and-betting-data?select=spreadspoke_scores.csv

## Production Database
The data will be loaded into a postgres relational database.

In [1]:
# import dependencies
import pandas as pd
import numpy as np

## Games Stats Data

## Extract Data
* read_stats - reads game stats and returns a data frame
* clean_columns - extracts the columns from each data frame used for the database and renames the columns

In [2]:
def read_stats(file_name):
    football_df = pd.read_csv(file_name)
    return football_df

In [3]:
def clean_columns(df):
    new_df = df[['Week','HomeTeam','AwayTeam','H-Score','A-Score']].sort_values('Week')
    r_df = new_df.rename(columns={'Week':'week','HomeTeam':'hometeam','AwayTeam':'awayteam','H-Score':'homescore','A-Score':'awayscore'})
    return r_df

## Transform and Clean for Loading
* create list of years. List corresponds to the csv data sources downloaded from Kaggle.
* iterate list, calls clean_columns, inserts *'year'* column and appends to lst_dfs.
* call pandas concat to create one data frame.
* transform year column to int64 and let pandas determine the data types for each column.
* drop index and save to final csv file without the pandas index column.

**Note**: each game stats file is saved individually (clean_game_stats_yyyy.csv) for debugging, if necessary.

In [4]:
years = ['2010','2011','2012','2013','2014','2015','2016','2017','2018','2019']
lst_dfs = []
for year in years:
    football_df = read_stats(f"game_stats_{year}.csv")
    new_df = clean_columns(football_df)
    new_df.insert(0,'year',year)
    lst_dfs.append(new_df)
    new_df.to_csv(f'./support_data/clean_game_stat_{year}.csv')
ret_df = pd.concat(lst_dfs)
ret_df = ret_df.astype({'year': 'int64'})
ret_df = ret_df.convert_dtypes()
ret_df = ret_df.reset_index(drop=True)
ret_df.to_csv('clean_game_stats_2010_2019.csv',index=False)

## Confirm data types are correct.

In [5]:
ret_df.dtypes

year          Int64
week          Int64
hometeam     string
awayteam     string
homescore     Int64
awayscore     Int64
dtype: object

## Scores and Betting Data
Data file contains information from 1966 - 2022.  A subset of this data (2010 - 2019) will be used for this database.
Since this is only one data file no methods were created. The data will be extracted and transformed in the below steps.
* Step 1 - Read data from csv into initial dataframe.

In [6]:
spreads = pd.read_csv('spreadspoke_scores.csv')

* Step 2 - Create subset (2010 - 2019)

In [7]:
spread_2010_2019_df = spreads[(spreads['schedule_season']>2009) & (spreads['schedule_season']<2020) & (spreads['schedule_playoff'] == False)]

* Step 3 - Extract the columns to be used for the database and rename the columns for consistency.

In [8]:
new_2010_2019_df = spread_2010_2019_df[['schedule_season','schedule_week','team_home','team_away','team_favorite_id','spread_favorite','over_under_line']]

In [9]:
new_sprd_df = new_2010_2019_df.rename(columns={'schedule_season':'year','schedule_week':'week','team_home':'hometeam','team_away':'awayteam',
                              'team_favorite_id':'teamfavid','spread_favorite':'pointspread','over_under_line':'outotal'})
new_sprd_df

,year,week,hometeam,awayteam,teamfavid,pointspread,outotal
10008,2010,1,New Orleans Saints,Minnesota Vikings,NO,-5.0,49.5
10009,2010,1,Buffalo Bills,Miami Dolphins,MIA,-3.0,39
10010,2010,1,Chicago Bears,Detroit Lions,CHI,-6.5,45
10011,2010,1,Houston Texans,Indianapolis Colts,IND,-1.0,48
10012,2010,1,Jacksonville Jaguars,Denver Broncos,JAX,-3.0,41.5
...,...,...,...,...,...,...,...
12662,2019,17,Minnesota Vikings,Chicago Bears,CHI,-4.5,36
12663,2019,17,New England Patriots,Miami Dolphins,NE,-17.0,45.5
12664,2019,17,New York Giants,Philadelphia Eagles,PHI,-3.5,44.5
12665,2019,17,Seattle Seahawks,San Francisco 49ers,SF,-3.5,47


* Step 4 - Add two columns used to join the two data sets.

In [10]:
new_sprd_df.insert(3, 'teamhomeid', 'team_home')
new_sprd_df.insert(5, 'teamawayid', 'team_away')

* Step 5 - Extract the unique team abbreviations from the game stats database and the unique teams from spread database.

In [11]:
teams_abbr = np.unique(ret_df[['hometeam']].values)
teams_abbr

array(['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL',
       'DEN', 'DET', 'GNB', 'HOU', 'IND', 'JAX', 'KAN', 'LAC', 'LAR',
       'MIA', 'MIN', 'NOR', 'NWE', 'NYG', 'NYJ', 'OAK', 'PHI', 'PIT',
       'SDG', 'SEA', 'SFO', 'STL', 'TAM', 'TEN', 'WAS'], dtype=object)

In [12]:
sprd_teams = np.unique(new_sprd_df[['hometeam']].values)
sprd_teams

array(['Arizona Cardinals', 'Atlanta Falcons', 'Baltimore Ravens',
       'Buffalo Bills', 'Carolina Panthers', 'Chicago Bears',
       'Cincinnati Bengals', 'Cleveland Browns', 'Dallas Cowboys',
       'Denver Broncos', 'Detroit Lions', 'Green Bay Packers',
       'Houston Texans', 'Indianapolis Colts', 'Jacksonville Jaguars',
       'Kansas City Chiefs', 'Los Angeles Chargers', 'Los Angeles Rams',
       'Miami Dolphins', 'Minnesota Vikings', 'New England Patriots',
       'New Orleans Saints', 'New York Giants', 'New York Jets',
       'Oakland Raiders', 'Philadelphia Eagles', 'Pittsburgh Steelers',
       'San Diego Chargers', 'San Francisco 49ers', 'Seattle Seahawks',
       'St. Louis Rams', 'Tampa Bay Buccaneers', 'Tennessee Titans',
       'Washington Redskins'], dtype=object)

* Step 6 - Create a dictionary using the unique team names/abbreviations from the two data sets.

In [13]:
team_array = ['Arizona Cardinals', 'Atlanta Falcons', 'Baltimore Ravens',
       'Buffalo Bills', 'Carolina Panthers', 'Chicago Bears',
       'Cincinnati Bengals', 'Cleveland Browns', 'Dallas Cowboys',
       'Denver Broncos', 'Detroit Lions', 'Green Bay Packers',
       'Houston Texans', 'Indianapolis Colts', 'Jacksonville Jaguars',
       'Kansas City Chiefs', 'Los Angeles Chargers', 'Los Angeles Rams',
       'Miami Dolphins', 'Minnesota Vikings', 'New England Patriots',
       'New Orleans Saints', 'New York Giants', 'New York Jets',
       'Oakland Raiders', 'Philadelphia Eagles', 'Pittsburgh Steelers',
       'San Diego Chargers', 'San Francisco 49ers', 'Seattle Seahawks',
       'St. Louis Rams', 'Tampa Bay Buccaneers', 'Tennessee Titans',
       'Washington Redskins']
team_ids = ['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL',
       'DEN', 'DET', 'GNB', 'HOU', 'IND', 'JAX', 'KAN', 'LAC', 'LAR',
       'MIA', 'MIN', 'NWE', 'NOR',  'NYG', 'NYJ', 'OAK', 'PHI', 'PIT',
       'SDG','SFO','SEA', 'STL','TAM', 'TEN', 'WAS']
len(team_ids)
team_dict = dict(zip(team_array, team_ids))
team_dict

{'Arizona Cardinals': 'ARI',
 'Atlanta Falcons': 'ATL',
 'Baltimore Ravens': 'BAL',
 'Buffalo Bills': 'BUF',
 'Carolina Panthers': 'CAR',
 'Chicago Bears': 'CHI',
 'Cincinnati Bengals': 'CIN',
 'Cleveland Browns': 'CLE',
 'Dallas Cowboys': 'DAL',
 'Denver Broncos': 'DEN',
 'Detroit Lions': 'DET',
 'Green Bay Packers': 'GNB',
 'Houston Texans': 'HOU',
 'Indianapolis Colts': 'IND',
 'Jacksonville Jaguars': 'JAX',
 'Kansas City Chiefs': 'KAN',
 'Los Angeles Chargers': 'LAC',
 'Los Angeles Rams': 'LAR',
 'Miami Dolphins': 'MIA',
 'Minnesota Vikings': 'MIN',
 'New England Patriots': 'NWE',
 'New Orleans Saints': 'NOR',
 'New York Giants': 'NYG',
 'New York Jets': 'NYJ',
 'Oakland Raiders': 'OAK',
 'Philadelphia Eagles': 'PHI',
 'Pittsburgh Steelers': 'PIT',
 'San Diego Chargers': 'SDG',
 'San Francisco 49ers': 'SFO',
 'Seattle Seahawks': 'SEA',
 'St. Louis Rams': 'STL',
 'Tampa Bay Buccaneers': 'TAM',
 'Tennessee Titans': 'TEN',
 'Washington Redskins': 'WAS'}

* Step 7 - Create a copy of the spread df.

In [14]:
upd_team_id = new_sprd_df.copy()

* Step 8 - Iterate the team_dict and update the two inserted columns with the team abbreviations.

In [15]:
for x, y in team_dict.items():
    upd_team_id.loc[upd_team_id['hometeam'] == x,'teamhomeid'] = y
    upd_team_id.loc[upd_team_id['awayteam'] == x,'teamawayid'] = y

* Step 8 - Update two columns to the correct data type.

In [16]:
upd_team_id = upd_team_id.astype({'outotal': 'float64', 'week': 'int64'})
upd_team_id

,year,week,hometeam,teamhomeid,awayteam,teamawayid,teamfavid,pointspread,outotal
10008,2010,1,New Orleans Saints,NOR,Minnesota Vikings,MIN,NO,-5.0,49.5
10009,2010,1,Buffalo Bills,BUF,Miami Dolphins,MIA,MIA,-3.0,39.0
10010,2010,1,Chicago Bears,CHI,Detroit Lions,DET,CHI,-6.5,45.0
10011,2010,1,Houston Texans,HOU,Indianapolis Colts,IND,IND,-1.0,48.0
10012,2010,1,Jacksonville Jaguars,JAX,Denver Broncos,DEN,JAX,-3.0,41.5
...,...,...,...,...,...,...,...,...,...
12662,2019,17,Minnesota Vikings,MIN,Chicago Bears,CHI,CHI,-4.5,36.0
12663,2019,17,New England Patriots,NWE,Miami Dolphins,MIA,NE,-17.0,45.5
12664,2019,17,New York Giants,NYG,Philadelphia Eagles,PHI,PHI,-3.5,44.5
12665,2019,17,Seattle Seahawks,SEA,San Francisco 49ers,SFO,SF,-3.5,47.0


## Confirm data types are correct.

In [17]:
upd_team_id = upd_team_id.convert_dtypes()
upd_team_id.dtypes

year             Int64
week             Int64
hometeam        string
teamhomeid      string
awayteam        string
teamawayid      string
teamfavid       string
pointspread    float64
outotal        float64
dtype: object

* Step 9 - Perform manual operations to clean the final data.

In [18]:
upd_team_id.reset_index(drop=True)

,year,week,hometeam,teamhomeid,awayteam,teamawayid,teamfavid,pointspread,outotal
0,2010,1,New Orleans Saints,NOR,Minnesota Vikings,MIN,NO,-5.0,49.5
1,2010,1,Buffalo Bills,BUF,Miami Dolphins,MIA,MIA,-3.0,39.0
2,2010,1,Chicago Bears,CHI,Detroit Lions,DET,CHI,-6.5,45.0
3,2010,1,Houston Texans,HOU,Indianapolis Colts,IND,IND,-1.0,48.0
4,2010,1,Jacksonville Jaguars,JAX,Denver Broncos,DEN,JAX,-3.0,41.5
...,...,...,...,...,...,...,...,...,...
2555,2019,17,Minnesota Vikings,MIN,Chicago Bears,CHI,CHI,-4.5,36.0
2556,2019,17,New England Patriots,NWE,Miami Dolphins,MIA,NE,-17.0,45.5
2557,2019,17,New York Giants,NYG,Philadelphia Eagles,PHI,PHI,-3.5,44.5
2558,2019,17,Seattle Seahawks,SEA,San Francisco 49ers,SFO,SF,-3.5,47.0


* Step 9a - Check if the teamfavid is equal to either the teamhomeid or teamawayid.

In [19]:
team_fav_df = upd_team_id.copy()
team_fav_df
not_eq = team_fav_df.loc[(team_fav_df['teamfavid'] != team_fav_df['teamhomeid']) & (team_fav_df['teamfavid'] != team_fav_df['teamawayid'])]

* Step 9b - Save rows that are not equal to csv for review.

In [20]:
not_eq.to_csv('./support_data/not_eq.csv')

* Step 9c - Get unique values for the rows in question.

In [21]:
tm_fav_u = np.unique(not_eq[['teamfavid']].values)

In [22]:
tm_fav_u

array(['GB', 'KC', 'LAC', 'LAR', 'LVR', 'NE', 'NO', 'PICK', 'SF', 'TB'],
      dtype=object)

* Step 9d - For GB, KC, LVR, NE, NO, SF, and TB change the teamfavid only.

In [35]:
team_fav_df
team_fav_df.loc[team_fav_df['teamfavid'] == 'GB','teamfavid'] = 'GNB'

* Step 9e - For LAC, LAR change the teamfavid based on the teamhomeid and teamawayid columns.

**Notes**:
* LAC and LAR moved to different locations.
* PICK is OK.  There was no favorite in this game.

In [39]:
team_fav_df.loc[upd_team_id['teamhomeid'] == 'SDG','teamfavid'] = 'SDG'
team_fav_df.loc[upd_team_id['teamawayid'] == 'SDG','teamfavid'] = 'SDG'

* Step 10 - Save final csv without pandas index.

In [40]:
team_fav_df.to_csv('spread_info_2010_2019.csv',index=False)

* Step 11 - Read in final csv.

In [ ]:
final_sprd_df = pd.read_csv('spread_info_2010_2019.csv')

In [44]:
final_sprd_df.head()

,year,week,hometeam,teamhomeid,awayteam,teamawayid,teamfavid,pointspread,outotal
0,2010,1,New Orleans Saints,NOR,Minnesota Vikings,MIN,NOR,-5.0,49.5
1,2010,1,Buffalo Bills,BUF,Miami Dolphins,MIA,MIA,-3.0,39.0
2,2010,1,Chicago Bears,CHI,Detroit Lions,DET,CHI,-6.5,45.0
3,2010,1,Houston Texans,HOU,Indianapolis Colts,IND,IND,-1.0,48.0
4,2010,1,Jacksonville Jaguars,JAX,Denver Broncos,DEN,JAX,-3.0,41.5


## Load Data
### Connect to database

In [51]:
import sqlalchemy
from sqlalchemy import create_engine
from sql_config import protocol, username,password,host,port,database_name

rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [46]:
final_sprd_df.to_sql(name='gamespreads', con=engine, schema='football_sch', if_exists='append', index=False)

In [47]:
ret_df.to_sql(name='gamestats', con=engine, schema='football_sch', if_exists='append', index=False)

## Confirm Data has been loaded

In [52]:
pd.read_sql_query('select * from football_sch.gameresults where year=2017', con=engine).head()

,year,week,hometeam,awayteam,homescore,awayscore,teamfavid,pointspread,outotal
0,2017,1,BUF,NYJ,21,12,BUF,-9.5,40.0
1,2017,1,CHI,ATL,17,23,ATL,-7.0,49.5
2,2017,1,CIN,BAL,0,20,CIN,-3.0,42.5
3,2017,1,CLE,PIT,18,21,PIT,-9.0,47.0
4,2017,1,DAL,NYG,19,3,DAL,-3.5,47.5


In [54]:
pd.read_sql_query("select * from football_sch.gameresults where hometeam='PHI' or awayteam='PHI'", con=engine).head()

,year,week,hometeam,awayteam,homescore,awayscore,teamfavid,pointspread,outotal
0,2010,1,PHI,GNB,20,27,GNB,-3.0,48.0
1,2010,2,DET,PHI,32,35,PHI,-6.5,41.0
2,2010,3,JAX,PHI,3,28,PHI,-3.0,45.0
3,2010,4,PHI,WAS,12,17,PHI,-5.0,43.5
4,2010,5,SFO,PHI,24,27,SFO,-3.0,38.5


In [50]:
engine.dispose()